In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tifffile

## Data Loading


In [ ]:
base_path = '/kaggle/input/hubmap-kidney-segmentation/'
print(os.listdir(base_path))

In [ ]:
#load the training data
train = pd.read_csv(base_path+'train.csv')
train.head()

In [ ]:
#load the saple submssion
ss = pd.read_csv(base_path+'sample_submission.csv')
ss.head()

In [ ]:
print(f'Number of train images: {train.shape[0]}')
print(f'Number of test images: {ss.shape[0]}')

In [ ]:
#Loading the additioanl information regarding the image files suchas patient id, targe annotation paths etc
info = pd.read_csv(base_path + 'HuBMAP-20-dataset_information.csv')
info.head(10)

## Sample Image Exploration

In [ ]:
#Load one of the tiff images
sample_img = tifffile.imread(base_path + "train/0486052bb.tiff")
plt.figure(figsize=(16, 16))
plt.imshow(sample_img);

In [ ]:
print('Sample Image Size: ', sample_img.shape)

In [ ]:
#Get the masks, viz. those points with 0 as their pixel value
mask = np.zeros_like(sample_img, dtype=np.bool)[:, :, 0].T.reshape(-1)

#Extract the annotation
annotation = train[train["id"] == "0486052bb"]["encoding"].values[0]

#Since its an str object, split it and convert its dtype to int
annotation = list(map(int, annotation.split()))

In [ ]:
for i in range(0, len(annotation), 2):
    mask[annotation[i] - 1 : annotation[i] - 1 + annotation[i + 1]] = 1

In [ ]:
#Visualise the mask
mask = mask.reshape(sample_img[:, :, 0].T.shape).T
plt.figure(figsize=(16, 16))
plt.imshow(mask);

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(sample_img[5000:15000, 5000:10000, :])
plt.imshow(mask[5000:15000, 5000:10000], alpha=0.5)
plt.title('Overalyed version of mask on the image');

In [ ]:
pd.read_json(base_path + "train/0486052bb-anatomical-structure.json")

In [ ]:
pd.read_json(base_path + "train/0486052bb.json")

In [ ]:
#splitting the information table for train and test images

info['split'] = 'test'
info.loc[info["image_file"].isin(os.listdir(base_path + "train")), "split"] = "train"

In [ ]:
info.head(2)

## EDA

In [ ]:
sns.countplot(info['sex']);

In [ ]:
sns.countplot(info['laterality']);

In [ ]:
sns.countplot(info['ethnicity']);

In [ ]:
sns.countplot(info['age']);

In [ ]:
sns.distplot(info['weight_kilograms']);

In [ ]:
sns.distplot(info['height_centimeters']);

In [ ]:
sns.distplot(info['bmi_kg/m^2']);

In [ ]:
sns.countplot(info['percent_cortex']);

In [ ]:
sns.countplot(info['percent_medulla']);